In [4]:
!pip install ultralytics torch torchvision opencv-python-headless albumentations optuna plotly seaborn scikit-learn pandas numpy matplotlib tqdm pyyaml pillow pathlib2 psutil wandb


In [5]:
# =====================================================================================
#         OPTIMIZED YOLO TRAINING SCRIPT (FINAL GPU VERSION)
# =====================================================================================

# Standard library imports
import os
import sys
import yaml
import json
from pathlib import Path

# Third-party imports
import cv2
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.utils.loss import v8DetectionLoss
# ✅ FIXED: Import the correct trainer class
from ultralytics.models.yolo.detect.train import DetectionTrainer
from tqdm import tqdm
import matplotlib.pyplot as plt

# =====================================================================================
# 🔧 1. MASTER CONFIGURATION
# =====================================================================================

# ===> DATASET PATH CONFIGURATION <===
# 🚨 IMPORTANT: Make sure your Google Drive is mounted in Colab.
# These paths should point to your dataset folders within Google Drive.
TRAIN_DATA_ROOT = Path("/content/drive/MyDrive/MICROSOFT HACK/hackathon2_train_1")
TEST_DATA_ROOT = Path("/content/drive/MyDrive/MICROSOFT HACK/Hackathon2_test1")

# ===> EXECUTION CONFIGURATION <===
# 🚀 Change the MODE here to 'train', 'predict', or 'visualize'
EXECUTION_CONFIG = {
    'MODE': 'train'  # Options: 'train', 'predict', 'visualize'
}

# CLASS NAMES - 7 SAFETY OBJECTS
CLASS_NAMES = [
    'OxygenTank', 'NitrogenTank', 'FirstAidBox', 'FireAlarm',
    'SafetySwitchPanel', 'EmergencyPhone', 'FireExtinguisher'
]

# OPTIMIZED HYPERPARAMETERS FOR T4 GPU
TRAINING_CONFIG = {
    'epochs': 50,
    'mosaic': 1.0,
    'optimizer': 'AdamW',
    'momentum': 0.937,
    'lr0': 0.01,
    'lrf': 0.01,
    'batch': 16,        # ✅ Increased batch size for GPU
    'imgsz': 640,
    'patience': 25,
    'warmup_epochs': 3,
    'weight_decay': 0.0005,
    'single_cls': False,
    'amp': True,        # ✅ Enabled Mixed Precision for T4 speedup
    'device': 0         # ✅ Set to use GPU
}

# KNOWLEDGE DISTILLATION SETTINGS
KD_CONFIG = {
    'enable': True,
    'teacher_model': 'yolov8l.pt',
    'student_model': 'yolov8s.pt',
    'temperature': 4.0,
    'hard_weight': 0.7,
    'soft_weight': 0.3,
}

# =====================================================================================
# 🧠 2. KNOWLEDGE DISTILLATION IMPLEMENTATION
# =====================================================================================
class YoloKDLoss(nn.Module):
    def __init__(self, student_model, hard_weight, soft_weight, temperature):
        super().__init__()
        self.hard_weight = hard_weight
        self.soft_weight = soft_weight
        self.temperature = temperature
        self.native_loss = v8DetectionLoss(student_model)
        self.kl_div = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_preds, teacher_preds, batch):
        hard_loss = self.native_loss(student_preds, batch)
        soft_loss = 0.0
        if len(student_preds) > 0 and len(teacher_preds) > 0 and student_preds[-1] is not None and teacher_preds[-1] is not None:
            student_logits = student_preds[-1]
            teacher_logits = teacher_preds[-1].detach()
            student_soft = torch.log_softmax(student_logits / self.temperature, dim=-1)
            teacher_soft = torch.softmax(teacher_logits / self.temperature, dim=-1)
            soft_loss = self.kl_div(student_soft, teacher_soft) * (self.temperature ** 2)
        return self.hard_weight * hard_loss + self.soft_weight * soft_loss

# ✅ FIXED: Inherit from DetectionTrainer instead of BaseTrainer
class KnowledgeDistillationTrainer(DetectionTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize the teacher model
        self.teacher = YOLO(KD_CONFIG['teacher_model']).model
        self.teacher.eval()
        for param in self.teacher.parameters():
            param.requires_grad = False

    def get_loss(self, preds, batch):
        # Initialize loss function on the first call
        if not hasattr(self, 'kd_loss'):
            self.kd_loss = YoloKDLoss(self.model, KD_CONFIG['hard_weight'], KD_CONFIG['soft_weight'], KD_CONFIG['temperature'])

        # Ensure teacher is on the correct device
        if next(self.teacher.parameters()).device != self.device:
            self.teacher = self.teacher.to(self.device)

        # Get teacher predictions and calculate loss
        with torch.no_grad():
            teacher_preds = self.teacher(batch['img'])
        return self.kd_loss(preds, teacher_preds, batch)

# =====================================================================================
# 📝 3. DATASET & UTILITY FUNCTIONS
# =====================================================================================
def create_optimized_yaml(data_path="yolo_params.yaml"):
    train_path = TRAIN_DATA_ROOT / "train_1" / "train1" / "images"
    val_path = TRAIN_DATA_ROOT / "train_1" / "val1" / "images"
    test_path = TEST_DATA_ROOT / "test1" / "images"

    for p in [train_path, val_path, test_path]:
        if not p.exists():
            raise FileNotFoundError(f"The path '{p}' does not exist. Please check your DATASET PATH CONFIGURATION.")

    yaml_config = {
        'train': str(train_path.resolve()), 'val': str(val_path.resolve()), 'test': str(test_path.resolve()),
        'nc': len(CLASS_NAMES), 'names': CLASS_NAMES
    }
    yaml_path = Path.cwd() / data_path
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_config, f, default_flow_style=False, sort_keys=False)
    print(f"✅ Dataset YAML created: {yaml_path}")
    return str(yaml_path)

# =====================================================================================
# 🚀 4. CORE FUNCTIONS (TRAIN, PREDICT, VISUALIZE)
# =====================================================================================
def train_enhanced_model():
    data_yaml = create_optimized_yaml()
    print("🚀 STARTING ENHANCED YOLO TRAINING")
    print("-" * 60)

    # Unpack the dictionary into the arguments
    train_args = {
        'model': KD_CONFIG['student_model'],
        'data': data_yaml,
        'project': 'runs/detect',
        'name': 'enhanced_safety_detection',
        'exist_ok': True,
        'verbose': True,
        'plots': True,
        **TRAINING_CONFIG
    }

    if KD_CONFIG['enable']:
        print(f"🧠 Training with Knowledge Distillation...")
        trainer = KnowledgeDistillationTrainer(overrides=train_args)
        trainer.train()
    else:
        print("💪 Starting Standard Training...")
        model = YOLO(KD_CONFIG['student_model'])
        model.train(**train_args)
    print("✅ Training complete!")

def run_enhanced_prediction():
    print("🔍 STARTING PREDICTION & EVALUATION")
    print("-" * 60)
    yaml_path = Path.cwd() / 'yolo_params.yaml'
    if not yaml_path.exists():
        raise FileNotFoundError("yolo_params.yaml not found. Please run training first.")
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    images_dir = Path(data['test'])

    runs_dir = Path.cwd() / "runs" / "detect"
    latest_run = max(runs_dir.glob('*'), key=os.path.getmtime)
    model_path = latest_run / "weights" / "best.pt"
    if not model_path.exists():
        raise FileNotFoundError(f"Could not find 'best.pt' in the latest run folder: {latest_run}")

    print(f"🤖 Loading model: {model_path}")
    model = YOLO(model_path)

    output_dir = Path.cwd() / "enhanced_predictions"; output_dir.mkdir(exist_ok=True)
    image_files = [f for f in images_dir.glob('*') if f.suffix.lower() in ['.png', '.jpg', '.jpeg']]

    for img_path in tqdm(image_files, desc="Predicting on test images"):
        model.predict(img_path, save=True, save_txt=True, project=str(output_dir), name='images', exist_ok=True, verbose=False)

    print(f"✅ Predictions saved to: {output_dir / 'images'}")
    print("\n📊 Running final evaluation on the test set...")
    metrics = model.val(data=str(yaml_path), split="test", device=TRAINING_CONFIG['device'])
    print("\n📈 EVALUATION RESULTS:")
    print(f"   mAP@0.5: {metrics.box.map50:.4f}")
    print(f"   mAP@0.5:0.95: {metrics.box.map:.4f}")

class EnhancedYoloVisualizer:
    MODE_TRAIN, MODE_VAL = 0, 1
    def __init__(self, train_root, val_root, class_names):
        self.train_root = Path(train_root); self.val_root = Path(val_root)
        self.classes = {i: name for i, name in enumerate(class_names)}
        self.set_mode(self.MODE_TRAIN)
        print("🎨 Visualizer Initialized.")
    def set_mode(self, mode):
        folder = self.train_root if mode == self.MODE_TRAIN else self.val_root
        self.mode_name = "TRAIN" if mode == self.MODE_TRAIN else "VALIDATION"
        self.images_folder = folder / "images"; self.labels_folder = folder / "labels"
        if not self.images_folder.exists(): raise FileNotFoundError(f"{self.mode_name} images not found at {self.images_folder}")
        self.image_names = sorted([f.name for f in self.images_folder.glob('*')])
        self.num_images = len(self.image_names); self.frame_index = 0
        print(f"✅ Mode: {self.mode_name} ({self.num_images} images)")
    def seek_frame(self, idx):
        if not (0 <= idx < self.num_images): return None
        img_path = self.images_folder / self.image_names[idx]
        lbl_path = self.labels_folder / img_path.with_suffix('.txt').name
        img = cv2.imread(str(img_path)); h, w = img.shape[:2]
        if lbl_path.exists():
            with open(lbl_path) as f:
                for line in f:
                    parts = line.split(); cid, x, y, cw, ch = int(parts[0]), *map(float, parts[1:5])
                    x1, y1 = int((x-cw/2)*w), int((y-ch/2)*h)
                    x2, y2 = int((x+cw/2)*w), int((y+ch/2)*h)
                    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
                    cv2.putText(img, self.classes[cid], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        return img
    def run(self):
        print("CONTROLS: [A] Prev | [D] Next | [T] Train | [V] Val | [Q] Quit")
        while True:
            frame = self.seek_frame(self.frame_index)
            # Use a dummy window if running in a non-GUI environment like Colab
            try:
                cv2.imshow("Visualizer", cv2.resize(frame, (1280, 720)))
                key = cv2.waitKey(0) & 0xFF
                if key == ord('q'): break
                elif key == ord('d'): self.frame_index = (self.frame_index + 1) % self.num_images
                elif key == ord('a'): self.frame_index = (self.frame_index - 1 + self.num_images) % self.num_images
                elif key == ord('t'): self.set_mode(self.MODE_TRAIN)
                elif key == ord('v'): self.set_mode(self.MODE_VAL)
            except cv2.error:
                print("⚠️ Could not display visualizer window. This is expected in environments like base Colab.")
                print("Visualization is intended for local execution with a GUI.")
                break
        cv2.destroyAllWindows()

# =====================================================================================
# 🚀 5. MAIN EXECUTION BLOCK
# =====================================================================================

try:
    mode = EXECUTION_CONFIG.get('MODE', 'train').lower()
    print("=" * 60)
    print(f"🚀 RUNNING IN MODE: {mode.upper()}")
    print("=" * 60)

    if mode == 'train':
        train_enhanced_model()
    elif mode == 'predict':
        run_enhanced_prediction()
    elif mode == 'visualize':
        train_root = TRAIN_DATA_ROOT / "train_1" / "train1"
        val_root = TRAIN_DATA_ROOT / "train_1" / "val1"
        vis = EnhancedYoloVisualizer(train_root, val_root, CLASS_NAMES)
        vis.run()
    else:
        print(f"❌ Invalid mode '{mode}'. Please choose 'train', 'predict', or 'visualize'.")

except Exception as e:
    import traceback
    print(f"\n❌ An error occurred: {e}")
    traceback.print_exc()

finally:
    print("\n🎉 Process finished.")

🚀 RUNNING IN MODE: TRAIN
✅ Dataset YAML created: /content/yolo_params.yaml
🚀 STARTING ENHANCED YOLO TRAINING
------------------------------------------------------------
🧠 Training with Knowledge Distillation...
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_params.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod